# OpenPNM Version 3:  The new @domain syntax 

In [1]:
import openpnm as op

Start by generating a simple 2D cubic network:

In [2]:
pn = op.network.Cubic([4, 4, 1])
print(pn)

――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――
openpnm.network.Cubic : net_01
――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――
#     Properties                                    Valid Values
――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――
1     pore.coords                                      16 / 16   
2     throat.conns                                     24 / 24   
――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――
#     Labels                                        Assigned Locations
――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――
1     pore.back                                     4         
2     pore.front                                    4         
3     pore.left                                     4         
4     pore.right                                    4         
5     pore.surface                                  12

This network includes a few pre-defined labels, which are boolean masks of ``True|False`` values which indicate whether that label applies to a given pore/throat or not.

We can use labels as masks to view only values for given locations, as follows:

In [3]:
pn['pore.coords'][pn['pore.left']]

array([[0.5, 0.5, 0.5],
       [0.5, 1.5, 0.5],
       [0.5, 2.5, 0.5],
       [0.5, 3.5, 0.5]])

or using the ``pores`` method:

In [4]:
pn['pore.coords'][pn.pores('left')]

array([[0.5, 0.5, 0.5],
       [0.5, 1.5, 0.5],
       [0.5, 2.5, 0.5],
       [0.5, 3.5, 0.5]])

In OpenPNM V3 there is a very handy new syntax, the @ symbol used as follows:

In [5]:
pn['pore.coords@left']

array([[0.5, 0.5, 0.5],
       [0.5, 1.5, 0.5],
       [0.5, 2.5, 0.5],
       [0.5, 3.5, 0.5]])

Using the @ symbol in this way is actually a side effect of a *major change* made in V3.  The ``Geometry`` and ``Physics`` objects are now *gone*.  There was essentially only one use case for these, which was to model heterogeneous domains, like bimodal pore size distributions or layered structures.  

In V2 this was accomplished by using 2 (or more) ``Geometry`` objects to represent each class of pores, with unique pore-scale models attached to each.  Without getting lost in the details, it is sufficient to say that having separate objects for managing each class of pores (and/or throats) created a *lot* of complications, both to the user and the to maintenence of the backend.  

In V3 we have developed what we think is a much tidier approach to managing heterogeneous domains.  Instead of creating multiple ``Geometry`` objects (and consequently multiple ``Physics`` objects), you now add all the pore-scale models to the ``Network`` object directly.  The trick is that when adding models you specify one additional argument: which pores (or throats) the model applies to, as follows:

In [6]:
pn.add_model(propname='pore.seed', 
             model=op.models.geometry.pore_seed.random,
             domain='left',
             seed=0,
             num_range=[0.2, 0.8])

where ``domain`` is a label which has already been defined and added to the network.  This means that to create a heterogeneous model you only need to create labels indicating each domain, the pass those labels when adding models.  You can also leave ``domain`` unspecified which means the model is applied everywhere.  For the above case, we can see that the 'pore.seed' model was computed for 4 locations (corresponding the 4 pores labelled 'left'):

In [9]:
print(pn)

――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――
openpnm.network.Cubic : net_01
――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――
#     Properties                                    Valid Values
――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――
1     pore.coords                                      16 / 16   
2     pore.seed                                         4 / 16   
3     throat.conns                                     24 / 24   
――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――
#     Labels                                        Assigned Locations
――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――
1     pore.back                                     4         
2     pore.front                                    4         
3     pore.left                                     4         
4     pore.right                                   